In [ ]:
import pandas as pd

# PART 1: Prepare the Data

In [ ]:
df = pd.read_csv("myopia.csv")

In [ ]:
df.info()

In [ ]:
df.sample(10)

In [ ]:
df["MYOPIC"].value_counts()

In [ ]:
df["AGE"].value_counts()

In [ ]:
df.isna().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
labels = df["MYOPIC"]
df_drop = df.drop(columns=["MYOPIC"])

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_data = scaler.fit_transform(df_drop)
scaled_data.shape

# PART 2: Apply Dimensionality Reduction

In [ ]:
from sklearn.decomposition import PCA

# Initialize PCA model

pca = PCA(n_components=0.90)

myopia_pca = pca.fit_transform(scaled_data)

In [ ]:
myopia_pca.shape # 14 features reduced to 10

In [ ]:
 # Transform PCA data to a DataFrame
    
df_myopia_pca = pd.DataFrame(
    data=myopia_pca,
    columns=["pc_1", "pc_2", "pc_3", "pc_4", "pc_5", "pc_6", "pc_7", "pc_8", "pc_9", "pc_10"]
)

df_myopia_pca.head()

In [ ]:
pca.explained_variance_ratio_

In [ ]:
pca.singular_values_

In [ ]:
from sklearn.manifold import TSNE

tsne_df = df_myopia_pca.copy()

 # Initialize t-SNE model
    
tsne = TSNE(learning_rate=10)

In [ ]:
 # Reduce dimensions
    
tsne_features = tsne.fit_transform(tsne_df)

In [ ]:
tsne_features.shape # Reduced 10 primary components down to 2

In [ ]:
import matplotlib.pyplot as plt

# Prepare to plot the dataset

# The first column of transformed features

tsne_df["x"] = tsne_features[:, 0]

# The second column of transformed features

tsne_df["y"] = tsne_features[:, 1]

 # Visualize the clusters
    
plt.scatter(tsne_df["x"], tsne_df["y"])
plt.show()

In [ ]:
labels.value_counts()

In [ ]:
 # Visualize the clusters with color
    
plt.scatter(tsne_df["x"], tsne_df["y"], c=labels)
plt.show()

# PART 3: Perform a Cluster Analysis with K-means

In [ ]:
 # Do this on Windows machines before importing KMeans to avoid a known bug (memory leak). 
    
import os
os.environ["OMP_NUM_THREADS"] = '1'

In [ ]:
from sklearn.cluster import KMeans

inertia = []

k = list(range(1,11))

# Looking for the best k

for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(df)
    inertia.append(km.inertia_)

# Define a DataFrame to plot the Elbow Curve using hvPlot

elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)

plt.plot(df_elbow["k"], df_elbow["inertia"])
plt.xticks(range(1,11))
plt.xlabel("Number of clusters")
plt.ylabel("Inertia")
plt.show()

In [ ]:
# Initializing model with K = 3 
    
model = KMeans(n_clusters=3, random_state=0)

model.fit(df)

In [ ]:
 # Get predictions
    
predictions = model.predict(df)

print(predictions)

In [ ]:
df["class"] = model.labels_
df.head()

In [ ]:
set(model.labels_)

In [ ]:
plt.scatter(df["SPHEQ"], df["ACD"], c=df["class"])
plt.xlabel("SPHEQ")
plt.ylabel("ACD")
plt.show()

# PART 4: Make a Recommendation

I'm not finding any discernable clusters from the different unsupervised learning algorithms we employed.